In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Задача 
- Реализуйте сверточную нейронную сеть с использованием tf.keras или keras. Используйте сверточные, пуллинговые и полносвязанные слои. Обучите на train выборке в течении 10 эпох, оцените качество на отложенной выборке
- Добавьте в предыдущую архитектуру слои BatchNorm. Обучите на train выборке в течении 10 эпох, оцените качество на отложенной выборке
- Создайте модель ResNet 50 (https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50), инициализированную случайными весами, и обучите ее на train выборке в течении 10 эпох, оцените качество на отложенной выборке
- Создайте модель ResNet 50, инициализированную весами ImageNet, и обучите ее на train выборке в течении 10 эпох, оцените качество на отложенной выборке

In [24]:
import numpy as np
import random
random.seed(10)
np.random.seed(0)

## 1. Реализуйте сверточную нейронную сеть с использованием tf.keras или keras. Используйте сверточные, пуллинговые и полносвязанные слои. Обучите на train выборке в течении 10 эпох, оцените качество на отложенной выборке

In [2]:
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
!ls /kaggle/input/hymenoptera-data/hymenoptera_data/train

ants  bees


In [4]:
train_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.) #загружает картинки на лету по мере необходимости, чтобы не держать их загруженными
test_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.) #1./255. значит что каждый пиксель делим на 255, то есть превращаем в число от 0 до 1

In [5]:
train = train_generator.flow_from_directory('/kaggle/input/hymenoptera-data/hymenoptera_data/train', target_size=(224, 224), batch_size=32, class_mode='binary') 
test = test_generator.flow_from_directory('/kaggle/input/hymenoptera-data/hymenoptera_data/val', target_size=(224, 224), batch_size=32, class_mode='binary')

Found 244 images belonging to 2 classes.
Found 153 images belonging to 2 classes.


In [6]:
i = keras.layers.Input(shape=(224,224,3)) 
o = keras.layers.Conv2D(16, (5,5))(i) 
o = keras.layers.MaxPool2D((2,2))(o)
o = keras.layers.ReLU()(o)
o = keras.layers.Conv2D(32, (3,3))(o)
o = keras.layers.MaxPool2D((2,2))(o)
o = keras.layers.ReLU()(o)
o = keras.layers.Conv2D(64, (3,3))(o)
o = keras.layers.MaxPool2D((2,2))(o)
o = keras.layers.ReLU()(o)
o = keras.layers.Flatten()(o)
o = keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(1e-4))(o)

In [28]:
model = keras.Model(inputs=i, outputs=o, name='ants_vs_bees')
model.compile(optimizer='adadelta', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "ants_vs_bees"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 110, 110, 16)      0         
_________________________________________________________________
re_lu_9 (ReLU)               (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 108, 108, 32)      4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
re_lu_10 (ReLU)              (None, 54, 54, 32)       

In [29]:
model.fit(train,
         epochs=10,
         validation_data=test)

Epoch 1/10
8/8 [==============================] - 3s 363ms/step - loss: 0.7082 - accuracy: 0.5820 - val_loss: 0.7135 - val_accuracy: 0.5163
Epoch 2/10
8/8 [==============================] - 2s 328ms/step - loss: 0.6920 - accuracy: 0.6357 - val_loss: 0.7130 - val_accuracy: 0.5163
Epoch 3/10
8/8 [==============================] - 2s 295ms/step - loss: 0.7069 - accuracy: 0.6028 - val_loss: 0.7125 - val_accuracy: 0.5229
Epoch 4/10
8/8 [==============================] - 2s 296ms/step - loss: 0.6874 - accuracy: 0.5979 - val_loss: 0.7120 - val_accuracy: 0.5359
Epoch 5/10
8/8 [==============================] - 2s 291ms/step - loss: 0.6560 - accuracy: 0.6912 - val_loss: 0.7114 - val_accuracy: 0.5359
Epoch 6/10
8/8 [==============================] - 2s 316ms/step - loss: 0.6688 - accuracy: 0.6500 - val_loss: 0.7109 - val_accuracy: 0.5425
Epoch 7/10
8/8 [==============================] - 2s 283ms/step - loss: 0.6590 - accuracy: 0.6696 - val_loss: 0.7104 - val_accuracy: 0.5425
Epoch 8/10
8/8 [====

## 2. Добавьте в предыдущую архитектуру слои BatchNorm. Обучите на train выборке в течении 10 эпох, оцените качество на отложенной выборке

In [47]:
i = keras.layers.Input(shape=(224,224,3)) 
o = keras.layers.Conv2D(16, (5,5))(i) 
o = keras.layers.MaxPool2D((2,2))(o)
o = keras.layers.ReLU()(o)
o = keras.layers.Conv2D(32, (3,3))(o)
o = keras.layers.MaxPool2D((2,2))(o)
o = keras.layers.ReLU()(o)
o = keras.layers.Conv2D(64, (3,3))(o)
o = keras.layers.MaxPool2D((2,2))(o)
o = keras.layers.ReLU()(o)
o = keras.layers.BatchNormalization()(o)
o = keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(1,
                         activation='sigmoid',
                         kernel_regularizer=tf.keras.regularizers.l1(1e-4))(o)

In [48]:
model = keras.Model(inputs=i, outputs=o)
model.compile(optimizer='adadelta', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
re_lu_33 (ReLU)              (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 108, 108, 32)      4640      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
re_lu_34 (ReLU)              (None, 54, 54, 32)        0   

In [49]:
model.fit(train,
         epochs=10,
         validation_data=test)

Epoch 1/10
8/8 [==============================] - 4s 449ms/step - loss: 0.8002 - accuracy: 0.4748 - val_loss: 0.7108 - val_accuracy: 0.6078
Epoch 2/10
8/8 [==============================] - 3s 393ms/step - loss: 0.7585 - accuracy: 0.5433 - val_loss: 0.7108 - val_accuracy: 0.5882
Epoch 3/10
8/8 [==============================] - 3s 412ms/step - loss: 0.7665 - accuracy: 0.5267 - val_loss: 0.7108 - val_accuracy: 0.6013
Epoch 4/10
8/8 [==============================] - 3s 384ms/step - loss: 0.7821 - accuracy: 0.5151 - val_loss: 0.7107 - val_accuracy: 0.5882
Epoch 5/10
8/8 [==============================] - 3s 376ms/step - loss: 0.7760 - accuracy: 0.5247 - val_loss: 0.7106 - val_accuracy: 0.5752
Epoch 6/10
8/8 [==============================] - 3s 329ms/step - loss: 0.7280 - accuracy: 0.5463 - val_loss: 0.7105 - val_accuracy: 0.5817
Epoch 7/10
8/8 [==============================] - 3s 368ms/step - loss: 0.7290 - accuracy: 0.5904 - val_loss: 0.7104 - val_accuracy: 0.5752
Epoch 8/10
8/8 [====

## 3. Создайте модель ResNet 50 (https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50), инициализированную случайными весами, и обучите ее на train выборке в течении 10 эпох, оцените качество на отложенной выборке

In [67]:
from tensorflow.keras.applications import ResNet50

In [68]:
base_model = ResNet50(weights=None,
                         include_top=False,
                         input_shape=(224, 224, 3))

In [53]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [70]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.layers[-3].output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1,
                         activation='sigmoid',
                         kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-4))(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x, name='resnet50_avsb')

In [71]:
model.compile(optimizer='adadelta', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [72]:
model.fit(train,
         epochs=10,
         validation_data=test)

Epoch 1/10
8/8 [==============================] - 7s 472ms/step - loss: 0.7318 - accuracy: 0.5365 - val_loss: 0.7321 - val_accuracy: 0.4706
Epoch 2/10
8/8 [==============================] - 3s 347ms/step - loss: 0.7321 - accuracy: 0.4857 - val_loss: 0.7321 - val_accuracy: 0.4706
Epoch 3/10
8/8 [==============================] - 3s 358ms/step - loss: 0.7319 - accuracy: 0.4922 - val_loss: 0.7321 - val_accuracy: 0.4575
Epoch 4/10
8/8 [==============================] - 3s 323ms/step - loss: 0.7321 - accuracy: 0.4871 - val_loss: 0.7321 - val_accuracy: 0.4575
Epoch 5/10
8/8 [==============================] - 3s 325ms/step - loss: 0.7316 - accuracy: 0.5208 - val_loss: 0.7321 - val_accuracy: 0.4575
Epoch 6/10
8/8 [==============================] - 3s 357ms/step - loss: 0.7317 - accuracy: 0.5013 - val_loss: 0.7321 - val_accuracy: 0.4575
Epoch 7/10
8/8 [==============================] - 3s 327ms/step - loss: 0.7316 - accuracy: 0.5100 - val_loss: 0.7321 - val_accuracy: 0.4575
Epoch 8/10
8/8 [====

## 4. Создайте модель ResNet 50, инициализированную весами ImageNet, и обучите ее на train выборке в течении 10 эпох, оцените качество на отложенной выборке

In [91]:
base_model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3))

In [92]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.layers[-3].output
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1,
                         activation='sigmoid',
                         kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-4))(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x, name='resnet50_avsb')

In [93]:
model.compile(optimizer='adadelta', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [94]:
model.fit(train,
         epochs=10,
         validation_data=test)

Epoch 1/10
8/8 [==============================] - 6s 464ms/step - loss: 0.7464 - accuracy: 0.4901 - val_loss: 0.7308 - val_accuracy: 0.5490
Epoch 2/10
8/8 [==============================] - 3s 361ms/step - loss: 0.7545 - accuracy: 0.4827 - val_loss: 0.7367 - val_accuracy: 0.5490
Epoch 3/10
8/8 [==============================] - 2s 320ms/step - loss: 0.7435 - accuracy: 0.4838 - val_loss: 0.7580 - val_accuracy: 0.5425
Epoch 4/10
8/8 [==============================] - 2s 311ms/step - loss: 0.7350 - accuracy: 0.5095 - val_loss: 0.7562 - val_accuracy: 0.5425
Epoch 5/10
8/8 [==============================] - 3s 375ms/step - loss: 0.7404 - accuracy: 0.5180 - val_loss: 0.7592 - val_accuracy: 0.5425
Epoch 6/10
8/8 [==============================] - 3s 360ms/step - loss: 0.7145 - accuracy: 0.5610 - val_loss: 0.7920 - val_accuracy: 0.5425
Epoch 7/10
8/8 [==============================] - 2s 318ms/step - loss: 0.7242 - accuracy: 0.5511 - val_loss: 0.7746 - val_accuracy: 0.5425
Epoch 8/10
8/8 [====